In [1]:
import itertools
import pandas as pd
import json
import networkx as nx
import ast
import time
from ast import literal_eval
import plotly.plotly as py
import plotly.graph_objs as go
from random import sample
from collections import defaultdict
import jupyterlab_dash
import dash
import dash_bootstrap_components as dbc
import dash_cytoscape as cyto
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import pickle

In [2]:
#load node and edge elements from pickled file
with open('elements.pkl', 'rb') as p:
    eles = pickle.load(p)


In [9]:
#rr = range(4501,9999)
#viewer = jupyterlab_dash.AppViewer(port=sample(rr,1)[0])
viewer = jupyterlab_dash.AppViewer()
roboto ='https://fonts.googleapis.com/css?family=Roboto'
poppins = 'https://fonts.googleapis.com/css?family=Poppins' #add Google Font as external stylesheet
ubuntu_condensed = 'https://fonts.googleapis.com/css?family=Poppins'

#create app instance
app = dash.Dash(__name__, external_stylesheets=['assets/app.css', roboto])

#create Flask dev server instance
server = app.server 

#create Cytoscape graph
graph = cyto.Cytoscape(
    id='cytoscape-graph',
    layout={'name': 'concentric'},
    style={'width':'100%', 'height': '600px'},
    elements=eles,
stylesheet=[
        # Group selectors
        {
            'selector': 'node',
            'style': {
                'background-color': '#00aced',
                'width':'data(width)',
                'height':'data(width)'
            }
        },    
    {'selector':'node:selected',
    'style': {'background-color':'#800080'}
    },

    {'selector':'.start-node',
        'style': {'background-color':'#800080'},
    },      
     {   
    "selector": "edge",
    "style": {
      "width": 5,
      "curve-style": "haystack",
      "line-color": "#999999",
      "line-style": "solid",
      "loop-direction": "-45deg",
      "loop-sweep": "-90deg",
      "arrow-scale": 5,
      "source-arrow-color": "#000000",
      "mid-source-arrow-color": "#999999",
      "target-arrow-color": "#fffff",
      "mid-target-arrow-color": "#ffffff",
      "source-arrow-shape": "none",
      "mid-source-arrow-shape": "none",
      "target-arrow-shape": "triangle-tee",
      "mid-target-arrow-shape": "triangle-tee",
      "source-arrow-fill": "filled",
      "mid-source-arrow-fill": "filled",
      "target-arrow-fill": "filled",
      "mid-target-arrow-fill": "filled"
    
        }
     }
])

d = []
for n in eles:
    try:
        if n['selected'] == True:
            d.append(n)
    except KeyError:
        pass
    
    
    
header = html.Header([], className='app-header')

body = html.Div([
    
                    #graph container
                    html.Div([dcc.Dropdown(id='dropdown-update-layout',
                style={'margin-right':'1em'},
                    value='grid',
                    clearable=False,
                    options=[
                            {'label': name.capitalize(), 'value': name}
                            for name in [ 'random', 'circle', 'cose','breadthfirst','concentric','grid']]),graph,],className='col-66',), 
             html.Div([html.H1("#rhetops Tweet Network"),
                   
                             
                        #tweet display container
                             html.Div([html.Div([html.H3('Tweet Information'),
                                                 html.P(['Tweet ID: ', html.A(d[0]['data']['tweet'], 
                                                  href=d[0]['data']['url'])]), 
                     html.P('Degree: {}'.format(d[0]['data']['degree'])), 
                     html.P('In-Degree: {}'.format(d[0]['data']['in_degree'])),
                     html.P('Out-Degree: {}'.format(d[0]['data']['out_degree'])),
                     html.H4('APA Citation:'),
                     html.Cite(d[0]['data']['citation'])
                     ],id='cytoscape-startNodeData', className='card'),
                    
                    html.Div(id='cytoscape-tapNodeData', className='card'),
                               
#                     dcc.Dropdown(
#                     id='dropdown-update-layout',
#                     value='grid',
#                     clearable=False,
#                     options=[
#                             {'label': name.capitalize(), 'value': name}
#                             for name in ['random', 'circle', 'cose','breadthfirst','concentric','grid']]),
                                                                     ])],className='col-33')],className="flex-container",)
                           
#footer = html.Div([html.Div()],className='footer')

#generate app
app.layout = html.Div([header,body])

# d = []
# for n in dat:
#     try:
#         if n['selected'] == True:
#             d.append(n)
#     except KeyError:
#         pass   
    
@app.callback(Output('cytoscape-graph', 'layout'),
              [Input('dropdown-update-layout', 'value'),])
def update_layout(layout):
    return {
        'name': layout,
        'animate': True
        }


@app.callback([Output('cytoscape-tapNodeData', 'children'),Output('cytoscape-startNodeData','style')],
              [Input('cytoscape-graph', 'tapNodeData')])
def displayTapNodeData(data):
    
    if data:
        return html.Div([html.P([html.H3('Tweet Information'),
                                 'Tweet ID: ', html.A(data['tweet'], 
                                                  href=data['url'])]), 
                     html.P('Degree: {}'.format(data['degree'])), 
                     html.P('In-Degree: {}'.format(data['in_degree'])),
                     html.P('Out-Degree: {}'.format(data['out_degree'])),
                     html.H4('APA Citation:'),
                     html.Cite(data['citation'])
                     ], style={'display':'block'}), {'display':'none'}  
    

    
    #return html.Div(html.A(data['id'], href=data['url']))
    
# @app.callback(Output('cytoscape-hoverNodeData', 'children'),
#               [Input('cytoscape-graph', 'tapNodeData')])
# def displayHoverNodeData(data):
#     html.P('Out-Degree: ' +str(data['out_degree']))
viewer.show(app)
# if __name__ == '__main__':
#     app.run_server(debug=False)

